<font color=blue size=6>Part 1. Autoencoder development</font>

![title](./octopusAI/autoencoder.png)

In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.layers import Reshape, Input
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


![title](./octopusAI/example0.png) ![title](./octopusAI/example1.png) ![title](./octopusAI/example2.png)
![title](./octopusAI/example3.png) ![title](./octopusAI/example4.png) ![title](./octopusAI/example5.png)
![title](./octopusAI/example6.png) ![title](./octopusAI/example7.png) ![title](./octopusAI/example8.png)
![title](./octopusAI/example9.png)

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [3]:
input_img = Input(shape=(28, 28, 1)) 


x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Reshape((392,))(x) 

encoded = Dense(32,activation='relu')(x)

x = Dense(392,activation='relu')(encoded)
x = Reshape((7,7,8))(x) 
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='relu', padding='same')(x)

In [4]:
import numpy as np
X_train = np.reshape(X_train, (len(X_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
X_test = np.reshape(X_test, (len(X_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

In [5]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [6]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 392)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                12576     
__________

In [7]:
from keras.callbacks import ModelCheckpoint
filepath="./octopusAI/model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
autoencoder.fit(X_train, X_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(X_test, X_test),
                callbacks=callbacks_list)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 63s 1ms/step - loss: 2107.2111 - val_loss: 1166.1915

Epoch 00001: val_loss improved from inf to 1166.19154, saving model to ./octopusAI/model.hdf5
Epoch 2/10
60000/60000 [==============================] - 63s 1ms/step - loss: 1049.2278 - val_loss: 924.0068

Epoch 00002: val_loss improved from 1166.19154 to 924.00677, saving model to ./octopusAI/model.hdf5
Epoch 3/10
60000/60000 [==============================] - 63s 1ms/step - loss: 888.8762 - val_loss: 821.0028

Epoch 00003: val_loss improved from 924.00677 to 821.00285, saving model to ./octopusAI/model.hdf5
Epoch 4/10
60000/60000 [==============================] - 63s 1ms/step - loss: 811.9210 - val_loss: 761.4447

Epoch 00004: val_loss improved from 821.00285 to 761.44475, saving model to ./octopusAI/model.hdf5
Epoch 5/10
60000/60000 [==============================] - 63s 1ms/step - loss: 762.8212 - val_loss: 722.1033

Epoch

In [9]:
inp = X_test[100]
inp.shape

(28, 28, 1)

In [10]:
inp3d = np.concatenate([inp, inp, inp], axis = 2)
inp3d.shape

(28, 28, 3)

In [11]:
inp4pred = inp.reshape((1,28,28,1))

In [12]:
from scipy import misc
misc.imsave('./octopusAI/real100.png', inp3d)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [13]:
res = autoencoder.predict(inp4pred)
print(res.shape)
res = res.reshape((28,28,1))
res = np.concatenate([res, res, res], axis = 2)
print(res.shape)
misc.imsave('./octopusAI/pred100.png', res)

(1, 28, 28, 1)
(28, 28, 3)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


![title](./octopusAI/real100.png)
![title](./octopusAI/pred100.png)

In [14]:
bottle_neck_model = Model(inputs=autoencoder.input, outputs=autoencoder.layers[6].output)

<font color=blue size=6>Part 2. Dataset for clustering</font>

In [15]:
X = bottle_neck_model.predict(X_train)

In [16]:
X.shape

(60000, 32)

In [17]:
X_TEST = bottle_neck_model.predict(X_test)

In [18]:
from keras.utils import np_utils
number_of_classes = 10
target_train = np_utils.to_categorical(y_train, number_of_classes)
target_test = np_utils.to_categorical(y_test, number_of_classes)

In [19]:
filepath="./octopusAI/convolutional_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [20]:
conv_net = Sequential()
conv_net.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
conv_net.add(MaxPooling2D(pool_size=(2, 2)))
conv_net.add(Conv2D(32, (3, 3), activation='relu'))
conv_net.add(MaxPooling2D(pool_size=(2, 2)))
conv_net.add(Dropout(0.2))
conv_net.add(Flatten())
conv_net.add(Dense(128, activation='relu'))
conv_net.add(Dense(number_of_classes, activation='softmax'))

In [21]:
conv_net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
conv_net.fit(X_train, target_train, validation_data=(X_test, target_test), epochs=5, batch_size=256, callbacks=callbacks_list)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 41s 685us/step - loss: 3.2204 - acc: 0.7469 - val_loss: 0.0967 - val_acc: 0.9696

Epoch 00001: val_acc improved from -inf to 0.96960, saving model to ./octopusAI/convolutional_model.hdf5
Epoch 2/5
60000/60000 [==============================] - 41s 676us/step - loss: 0.1167 - acc: 0.9647 - val_loss: 0.0644 - val_acc: 0.9795

Epoch 00002: val_acc improved from 0.96960 to 0.97950, saving model to ./octopusAI/convolutional_model.hdf5
Epoch 3/5
60000/60000 [==============================] - 40s 669us/step - loss: 0.0761 - acc: 0.9758 - val_loss: 0.0525 - val_acc: 0.9821

Epoch 00003: val_acc improved from 0.97950 to 0.98210, saving model to ./octopusAI/convolutional_model.hdf5
Epoch 4/5
60000/60000 [==============================] - 40s 671us/step - loss: 0.0634 - acc: 0.9799 - val_loss: 0.0473 - val_acc: 0.9842

Epoch 00004: val_acc improved from 0.98210 to 0.98420, saving model to ./

In [23]:
filepath="./octopusAI/feedforward_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [24]:
feed_net = Sequential()
feed_net.add(Dense(64, input_shape=(32,), activation='relu'))
feed_net.add(Dropout(0.2))
feed_net.add(Dense(64, activation='relu'))
feed_net.add(Dropout(0.2))
feed_net.add(Dense(64, activation='relu'))
feed_net.add(Dense(number_of_classes, activation='softmax'))

In [25]:
feed_net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
feed_net.fit(X, target_train, validation_data=(X_TEST, target_test), epochs=25, batch_size=256, callbacks=callbacks_list)

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 1s 21us/step - loss: 12.2442 - acc: 0.2344 - val_loss: 10.5002 - val_acc: 0.3444

Epoch 00001: val_acc improved from -inf to 0.34440, saving model to ./octopusAI/feedforward_model.hdf5
Epoch 2/25
60000/60000 [==============================] - 1s 14us/step - loss: 8.9163 - acc: 0.4355 - val_loss: 6.0879 - val_acc: 0.6123

Epoch 00002: val_acc improved from 0.34440 to 0.61230, saving model to ./octopusAI/feedforward_model.hdf5
Epoch 3/25
60000/60000 [==============================] - 1s 14us/step - loss: 5.8534 - acc: 0.6169 - val_loss: 3.3392 - val_acc: 0.7796

Epoch 00003: val_acc improved from 0.61230 to 0.77960, saving model to ./octopusAI/feedforward_model.hdf5
Epoch 4/25
60000/60000 [==============================] - 1s 15us/step - loss: 4.0499 - acc: 0.7219 - val_loss: 2.6451 - val_acc: 0.8190

Epoch 00004: val_acc improved from 0.77960 to 0.81900, saving model to ./octopusA

In [27]:
conv_net.load_weights("./octopusAI/convolutional_model.hdf5")
feed_net.load_weights("./octopusAI/feedforward_model.hdf5")

In [28]:
from sklearn.metrics import confusion_matrix

res_conv = conv_net.predict(X_test)
res_feedforward = feed_net.predict(X_TEST)

confusion_conv = confusion_matrix(target_test.argmax(axis=1), res_conv.argmax(axis=1))
confusion_feedforward = confusion_matrix(target_test.argmax(axis=1), res_feedforward.argmax(axis=1))

In [29]:
print(confusion_conv)
print('========================================================')
print(confusion_feedforward)

[[ 970    0    4    0    1    0    3    1    0    1]
 [   0 1128    1    1    0    0    4    0    1    0]
 [   1    0 1024    0    0    1    0    5    1    0]
 [   0    0    1 1005    0    1    0    1    2    0]
 [   0    0    0    0  974    0    3    0    0    5]
 [   1    0    0   13    0  876    1    1    0    0]
 [   3    1    0    0    2    4  945    0    1    2]
 [   0    4    4    3    0    0    0 1013    1    3]
 [   7    0    7    5    2    7    0    5  937    4]
 [   2    2    1    2    9    3    0    3    2  985]]
[[ 963    0    0    1    0    2    7    1    5    1]
 [   0 1114    7    2    2    1    3    0    6    0]
 [  11    0  975   14    5    0    5    9   13    0]
 [   2    0    4  959    0   12    1    5   21    6]
 [   0    0    1    0  943    0    4    3    1   30]
 [   3    1    0   24    2  836   12    1   10    3]
 [   4    3    0    2    2    4  939    0    4    0]
 [   2    5   12    2    5    0    0  951    0   51]
 [   5    0    4   22    5   10    3    2  90

<font color=blue size=6>Part 3. Clustering</font>

In [30]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=15).fit(X)

In [31]:
xt = kmeans.predict(X_TEST)

In [32]:
kmeans_confusion = np.zeros((10,10))

for i in range(0,len(y_test)):
    kmeans_confusion[y_test[i], xt[i]] +=1
    
kmeans_confusion = kmeans_confusion.astype(int)

print(kmeans_confusion)

[[ 64  72  32   1   3 695  13   1  97   2]
 [  9   2   5   0   2   0 333   3   4 777]
 [ 13 251  33   8 535  11  43   9  86  43]
 [108 203   5  12 137  22  13  17 468  25]
 [ 80  51  33 114   4   0  97 583   0  20]
 [265  67  13  29  14  35 154  32 271  12]
 [ 13 100 753   0   2  12  37   8   3  30]
 [ 26   9   0 663   5   1  46 235   0  43]
 [447 155   9  29   8   5  69  47 176  29]
 [ 51  21   0 323   1   5  10 575   7  16]]
